In [1]:
import django
import os

os.environ["DJANGO_SETTINGS_MODULE"] = 'imdb.settings'
django.setup()

In [2]:
from DatabaseQueries.models import *
from django.db.models import Subquery, OuterRef, Sum, FloatField
from django.db.models.functions import Cast

In [3]:
def get_top_10_movies_with_avg_rating():
    top_10_movies = MovieRating.objects.values('movie_id', 'movie__title').annotate(avg_rating=Cast(Sum(F('no_of_ratings') * F('rating_no') * 1.0) / Sum(F('no_of_ratings')), FloatField()))
    return top_10_movies.values('movie__title','avg_rating').order_by('-avg_rating')[:10]
get_top_10_movies_with_avg_rating()

<QuerySet [{'movie__title': 'H', 'avg_rating': 3.5}, {'movie__title': 'I', 'avg_rating': 3.4047619047619047}, {'movie__title': 'J', 'avg_rating': 3.2222222222222223}, {'movie__title': 'E', 'avg_rating': 3.1914893617021276}, {'movie__title': 'A', 'avg_rating': 3.1621621621621623}, {'movie__title': 'C', 'avg_rating': 3.025}, {'movie__title': 'G', 'avg_rating': 3.024390243902439}, {'movie__title': 'B', 'avg_rating': 2.9555555555555557}, {'movie__title': 'F', 'avg_rating': 2.85}, {'movie__title': 'K', 'avg_rating': 2.7906976744186047}]>

In [8]:
def get_top_5_and_least_5_actors():
    top_5_actors = Actor.objects.all().annotate(no_of_movies_acted=Count('moviecast')).order_by('-no_of_movies_acted').values('id')[:5]
    least_5_actors = Actor.objects.all().annotate(no_of_movies_acted=Count('moviecast')).order_by('no_of_movies_acted').values('id')[:5]
    no_of_movies_acted = Actor.objects.filter(Q(id__in=Subquery(top_5_actors)) | Q(id__in=Subquery(least_5_actors))).values_list('name')

    return no_of_movies_acted
get_top_5_and_least_5_actors()

<QuerySet [{'id': 1}, {'id': 11}, {'id': 12}, {'id': 2}, {'id': 3}]>
<QuerySet [{'id': 6}, {'id': 7}, {'id': 8}, {'id': 9}, {'id': 2}]>


<QuerySet [('Pavan Kalyan',), ('Prabhas',), ('Ram Charan',), ('Rashi',), ('Deepika',), ('Nithin',), ('Bala Krishna',), ('Samantha',), ('Tamanna',)]>

In [10]:
def get_top_5_and_least_5_actors():
    no_of_movies_acted = Actor.objects.annotate(no_of_movies_acted=Count('moviecast')).values_list('name', 'no_of_movies_acted')
    top_5_actors = no_of_movies_acted.order_by('-no_of_movies_acted')[:5]
    least_5_actors = no_of_movies_acted.order_by('no_of_movies_acted')[:5]
    result = {"top": top_5_actors, "least": least_5_actors}
    return result
get_top_5_and_least_5_actors()

{'top': <QuerySet [('Pavan Kalyan', 3), ('Rashi', 3), ('Deepika', 3), ('Prabhas', 2), ('Ram Charan', 2)]>,
 'least': <QuerySet [('Nithin', 0), ('Bala Krishna', 0), ('Samantha', 1), ('Tamanna', 1), ('Prabhas', 2)]>}